In [1]:
import torch
from nerfstudio.utils.eval_utils import eval_setup
import numpy as np
import cv2
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# 1. Load your pre-trained model
config_path = Path("YOUR_MODEL_PATH/config.yml")
_, pipeline, _, _ = eval_setup(config_path=config_path, test_mode="inference")

In [ ]:
# 2. Prepare your target image (real brain surface image)
target_image_path = "path/to/brain_surface_image.png"
target_image = cv2.imread(target_image_path)
target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2RGB)
target_image = torch.from_numpy(target_image).float() / 255.0  # Normalize to [0,1]

# Ensure target image has the right shape
if len(target_image.shape) == 3:
    # If target is HxWx3, we're good
    pass
else:
    # Reshape if needed
    target_image = target_image.reshape(512, 512, 3)  # Adjust dimensions as needed


In [ ]:
# 3. Set your initial transformation matrix (good starting point)
initial_transform = torch.tensor([
    [-0.16551750084030392, 0.17173068552312126, -0.9711398089695112, -80.73672706466026],
    [-0.9756847655052445, 0.11494663876681419, 0.18661861803471735, 15.51473463480075],
    [0.14367740002017088, 0.9784149640546033, 0.14852933325600515, 12.348141976499424],
    [0, 0, 0, 1],
], dtype=torch.float32)


In [ ]:
# 4. Run the optimization
aligner = BrainSurfaceAligner(
    pipeline=pipeline,
    target_image=target_image,
    initial_transform=initial_transform,
    learning_rate=0.005,  # You may need to tune this
    image_height=512,
    image_width=512,
    max_iterations=500,
)

# Optional visualization callback
def viz_callback(iteration, loss, image, transform):
    # Save current render
    if iteration % 50 == 0:
        img_np = (image.detach().cpu().numpy() * 255).astype(np.uint8)
        cv2.imwrite(f"iteration_{iteration}.png", cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR))
        print(f"Current transform:\n{transform}")

# Run optimization
results = aligner.run_optimization(callback=viz_callback)


In [ ]:
# 5. Use the optimized camera parameters
final_transform = results["final_transform"]
print(f"Final transformation matrix:\n{final_transform}")
print(f"Final loss: {results['final_loss']}")